In [ ]:
import respiration.utils as utils

device = utils.get_torch_device()
device

In [ ]:
import os
import torch
from respiration.extractor.efficient_phys import EfficientPhys

dim = 72
frame_depth = 20

# Wrap modul in nn.DataParallel
model = EfficientPhys(img_size=dim, frame_depth=frame_depth)
# Fix model loading: Some key have an extra 'module.' prefix
model = torch.nn.DataParallel(model)
model.to(device)

pretrained_model = os.path.join('..', 'data', 'rPPG-Toolbox', 'UBFC-rPPG_EfficientPhys.pth')
key_matching = model.load_state_dict(torch.load(pretrained_model, map_location=device))

## Dataset and dataloader

In [ ]:
import respiration.dataset as repository

dataset = repository.from_default()
scenarios_all = dataset.get_scenarios([
    '101_natural_lighting',
])

split_ratio = 0.8
training = scenarios_all[:int(len(scenarios_all) * split_ratio)]
testing = scenarios_all[int(len(scenarios_all) * split_ratio):]

testing

In [ ]:
from scipy.signal import resample
from torch.utils.data import Dataset


class RespirationDataset(Dataset):
    def __init__(self,
                 source: repository.Dataset,
                 scenarios: list[tuple[str, str]],
                 to: torch.device = torch.device('cpu')):
        self.dataset = source
        self.scenarios = scenarios
        self.device = to

    def __len__(self):
        return len(self.scenarios)

    def __getitem__(self, idx):
        subject, scenario = self.scenarios[idx]
        frames, _ = self.dataset.get_video_rgb(subject, scenario, False)
        gt_waveform, _ = self.dataset.get_ground_truth_rr_signal(subject, scenario)

        # The ground truth signal is sample with a higher frequency than the frames...
        gt_waveform = resample(gt_waveform, len(frames))

        # TODO: Find a better way to handle batching and chunking...
        chunk_size = frame_depth * 80 + 1

        gt_waveform = gt_waveform[:chunk_size]
        frames = frames[:chunk_size]

        # Down-sample the video frames to the desired dimension
        frames = utils.down_sample_video(frames, dim)

        # Create the frames tensor and the ground truth tensor
        frames = torch.tensor(frames,
                              dtype=torch.float32,
                              device=self.device).permute(0, 3, 1, 2)
        gt_waveform = torch.tensor(gt_waveform,
                                   dtype=torch.float32,
                                   device=self.device)

        return frames, gt_waveform


training_data = RespirationDataset(dataset, training, device)
test_data = RespirationDataset(dataset, testing, device)

In [ ]:
from torch.utils.data import DataLoader

training_loader = DataLoader(training_data, batch_size=1, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=True)

In [ ]:
# Display image and label.
#train_features, train_labels = next(iter(training_loader))
#print(f"Feature batch shape: {train_features.size()}")
#print(f"Labels batch shape: {train_labels.size()}")
#print(f"Device: {train_features.device}")

## Define loss function

In [ ]:
# Define the negative Pearson loss function
def pearson_loss(y_pred, y_true):
    mean_true = torch.mean(y_true)
    mean_pred = torch.mean(y_pred)
    num = torch.sum((y_true - mean_true) * (y_pred - mean_pred))
    den = torch.sqrt(torch.sum((y_true - mean_true) ** 2) * torch.sum((y_pred - mean_pred) ** 2))
    return -num / den

## Training

In [ ]:
learning_rate = 0.001
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
from torch.utils.tensorboard import SummaryWriter


def train_one_epoch(epoch_index: int, tb_writer: SummaryWriter):
    running_loss = 0.0
    last_loss = 0.0

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for idx, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        for idy in range(inputs.size(0)):
            # Make predictions for this batch
            outputs = model(inputs[idy])

            # Compute the loss and its gradients
            loss = pearson_loss(outputs, labels[idy])
            loss.backward()

            # Adjust learning weights
            optimizer.step()

            # Gather data and report
            running_loss += loss.item()

        last_loss = running_loss / 1000  # loss per batch
        print('  batch {} loss: {}'.format(idx + 1, last_loss))
        tb_x = epoch_index * len(training_loader) + idx + 1
        tb_writer.add_scalar('Loss/train', last_loss, tb_x)
        running_loss = 0.

    return last_loss

In [ ]:
from datetime import datetime

# Initializing in a separate cell, so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

model_output_dir = os.path.join('..', 'models', 'efficeint_phys_fine_tuned', timestamp)
if not os.path.exists(model_output_dir):
    os.makedirs(model_output_dir)

writer = SummaryWriter(os.path.join(model_output_dir, 'fashion_trainer'))

EPOCHS = 2

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print(f'EPOCH {epoch + 1}:')

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch, writer)
    print(f'  avg loss: {avg_loss}')

    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for inx, vdata in enumerate(test_loader):
            vinputs, vlabels = vdata

            for idy in range(vinputs.size(0)):
                voutputs = model(vinputs[idy])
                vloss = pearson_loss(voutputs, vlabels[idy])
                running_vloss += vloss

    avg_vloss = running_vloss / (inx + 1)
    print(f'LOSS train {avg_loss} valid {avg_vloss}')

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                       {'Training': avg_loss, 'Validation': avg_vloss},
                       epoch + 1)
    writer.flush()

    # Track the best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = os.path.join(model_output_dir, f'model_{timestamp}_{epoch}')
        torch.save(model.state_dict(), model_path)